<h2>Imports</h2>

In [2]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from glob import glob

<h2>Utils

In [3]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

<h2>Constants</h2>

In [4]:
datasetPath = "Data/dinoSparseRing/*.png"
ß1 = 2
ß2 = 32  
# We associate with p a reference image R(p),the images S(p) where p should be visible and the images T(p) where it is truly found  
patchModel = {
    "R":None,
    "S":set,
    "T":set
}
# The cell C(i, j) keeps track of two different sets Qt(i, j) and Qf(i, j)
cell = {
    "Qt":set,
    "Qf":set
}
# We associate with each image I a regular grid of β1×β1 pixel^2 cells
imageModel = {
    "img":np.ndarray,
    "grid":np.ndarray
}

<h2>Get Input

In [5]:
# Initialize image model from a given path
def init_imgs(datasetPath):
    # Read imgs
    imgs = [cv.rotate(cv.imread(file),cv.ROTATE_90_COUNTERCLOCKWISE) for file in glob(datasetPath)]
    # imgs = [cv.rotate(img,cv.ROTATE_90_COUNTERCLOCKWISE) for img in imgs]

    # Construct corresponding image grid
    grids = [np.array([np.array([cell for x in range(0,img.shape[1]//ß1)]) for y in range(0,img.shape[0]//ß1)]) for img in imgs]

    # print('images count:',len(imgs))
    # print('grids count:',len(grids))
    # print('image size:',imgs[0].shape)
    # print('grid size:',grids[0].shape)

    return imgs,grids

<h2>Feature Detection

In [6]:
# Get Harris and DoG operators for a given image
def get_harris_dog(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    
    '''
    To ensure uniform coverage, we lay over each image a coarse regular grid of
    ß2×ß2 pixel2 cells, and return as corners and blobs for each
    cell the η local maxima of the two operators with strongest
    responses (we use ß2 = 32 and η = 4 in all our experiments).
    '''

    # cells = np.zeros((gray.shape[0])//ß2,gray.shape[1])//ß2))
    # for vInc in range(0,gray.shape[0]-31):
    #     for hInc in range(0,gray.shape[0]-31):
    #         g1 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=1)
    #         g2 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=np.sqrt(2))
    #         diff = cv.absdiff(g1,g2)
    #         # dog = dog*(np.max(dog)/255)
    #         dog1 = diff * gray[vInc:vInc+31][hInc:hInc+31]
    #         g1 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=1)
    #         g2 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=np.sqrt(2))
    #         diff = cv.absdiff(g1,g2)
    #         # dog = dog*(np.max(dog)/255)
    #         dog2 = diff * gray[vInc:vInc+31][hInc:hInc+31]
    #         g1 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=1)
    #         g2 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=np.sqrt(2))
    #         diff = cv.absdiff(g1,g2)
    #         # dog = dog*(np.max(dog)/255)
    #         dog3 = diff * gray[vInc:vInc+31][hInc:hInc+31]
    #         g1 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=1)
    #         g2 = cv.GaussianBlur(gray[vInc:vInc+31][hInc:hInc+31],(0,0),sigmaX=np.sqrt(2))
    #         diff = cv.absdiff(g1,g2)
    #         # dog = dog*(np.max(dog)/255)
    #         dog4 = diff * gray[vInc:vInc+31][hInc:hInc+31]
    #         for i in range(dog1.shape[0]):
    #             for j in range(dog1.shape[1]):
    #                 if(dog1[i][j] != dog2[i][j] and dog1[i][j] != dog3[i][j] and dog1[i][j] != dog4[i][j] and dog2[i][j] != dog3[i][j] and dog2[i][j] != dog4[i][j] and dog3[i][j] != dog4[i][j]):
    #                     print("DIFFFF")
    #         return
            
    # Get DoG
    g1 = cv.GaussianBlur(gray,(0,0),sigmaX=1)
    g2 = cv.GaussianBlur(gray,(0,0),sigmaX=1*np.sqrt(2))
    diff = cv.absdiff(g1,g2)
    # dog = dog*(np.max(dog)/255)
    dog = diff * gray

    # Get Harris
    bSize = 3
    kSize = 1
    corners = cv.cornerHarris(src=gray,blockSize=bSize,ksize=kSize,k=0.06)
    corners = cv.dilate(corners,None)
    img[corners>0.01*corners.max()] = [0,0,255]

    show_images([img,diff,dog],['detected corners blockSize='+str(bSize)+' ksize='+str(kSize),'dog before multiplication of img','dog after multiplication of img'])
    
    return dog , corners

In [6]:
images,grids = init_imgs(datasetPath)
imagesFeatures = []
for idx,image in enumerate(images):
    dog,harris = get_harris_dog(image)
    imagesFeatures.append(
        {
            "image":images[idx],
            "grid":grids[idx],
            "dog":dog,
            "harris":harris
        }
    )

<h2>Trangulation

In [36]:
P1 = np.float32(np.array([
    [-0.14396457836077139000, 0.96965263281337499000, 0.19760617153779569000, -0.90366580603479685000],[-0.04743335255026152200, -0.42560419233334673000, -0.40331536459778505000, -0.23984130575212276000],[0.88306936201487163000, -0.010415508744, -0.0294278883669, 0.673097816109]
]))

P2 = np.float32(np.array([
    [-0.23143687262851170000, 0.96422332027030622000, 0.12926937165558827000, -0.65860811165749811000],[-0.05749426020475298400, -0.75028640486482034000, -0.71601208187197274000, -0.25878137856376976000],[0.64835049619606955000, -0.0143392676288,-0.0315356332107, 0.659314086504]
]))

p1 = np.float32([100,6])
p2 = np.float32([900,9])

print(cv.triangulatePoints(P1,P2,p1,p2))
# F = [
#     [3310.400000, 0.000000, 316.730000],
#     [0.000000, 3325.500000, 200.550000],
#     [0.000000, 0.000000, 1.000000]
# ]
# F = np.array(F)

# p1 = (5,6,1)
# p2 = (10,9,1)

# T1 = [
#     [1,0,-p1[0]],
#     [0,1,-p1[1]],
#     [0,0,1]
# ]
# T2 = [
#     [1,0,-p2[0]],
#     [0,1,-p2[1]],
#     [0,0,1]
# ]

# T2 = np.array(T2)
# T1 = np.array(T1)
# F = np.transpose(np.linalg.inv(T2))*F*np.linalg.inv(T1)

# plt.subplot(221)
# plt.scatter([p1[0]],[p1[1]],color='red')
# plt.subplot(222)
# plt.scatter([p2[0]],[p2[1]],color='red')

[[ 0.01782972]
 [ 0.6826026 ]
 [-0.72965467]
 [-0.03660343]]
